In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# --- 1. (8일차) Custom Dataset 클래스 ---
# (8일차 코드와 동일, __getitem__에서 label 모양만 수정)
class JeonseDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.features = df.drop('risk_label', axis=1).values
        self.labels = df['risk_label'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]

        # BCELoss를 위해 둘 다 float32로 변환
        feature_tensor = torch.tensor(feature, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        # 모델 출력 [batch_size, 1]과 모양을 맞추기 위해 .view(1)
        return feature_tensor, label_tensor.view(1)

# --- 2. (5일차) 모델(MLP) 클래스 ---
# (5일차 코드와 동일, 입력 feature 개수만 3개로 수정)
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        # layer1: 입력 3개(전세가율, 근저당, 건물연식)
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

# --- 3. (9일차 핵심) 학습 준비 ---

# (GPU 설정)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# (Hyperparameters 설정)
input_dim = 3       # Feature 개수 (전세가율, 근저당, 건물연식)
hidden_dim = 8      # 은닉층 뉴런 개수 (임의로 8로 설정)
output_dim = 1      # 출력 개수 (위험 확률 1개)
learning_rate = 0.001 # 학습률 (Adam은 0.001 정도가 무난)
batch_size = 16     # 배치 크기
num_epochs = 50     # 총 Epoch 수

# (데이터 준비)
csv_file_path = 'dummy_data.csv' # 8일차에 생성한 파일
dataset = JeonseDataset(csv_file_path)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# (모델, 손실함수, 옵티마이저 준비)
model = SimpleMLP(input_dim, hidden_dim, output_dim).to(device)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("--- 학습 시작 ---")

# --- 4. (9일차 핵심) 중첩 학습 루프 (Nested Loop) ---

# 바깥쪽 루프 (Epoch)
for epoch in range(num_epochs):

    # Epoch마다 평균 손실과 정확도를 계산하기 위한 변수
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    # 안쪽 루프 (Batch / Iteration)
    # data_loader가 16개씩 데이터를 묶어서 공급
    for features_batch, labels_batch in data_loader:

        # 1. 데이터를 GPU로 보냄
        features_batch = features_batch.to(device)
        labels_batch = labels_batch.to(device)

        # 2. 순전파 (Forward)
        prediction = model(features_batch)

        # 3. 손실 계산
        loss = loss_fn(prediction, labels_batch)

        # 4. 기울기 초기화
        optimizer.zero_grad()

        # 5. 역전파 (Backward)
        loss.backward()

        # 6. 파라미터 업데이트
        optimizer.step()

        # --- (Epoch 통계 계산) ---
        # 현재 배치의 손실을 누적
        epoch_loss += loss.item()

        # 현재 배치의 정확도 계산 (7일차 복습)
        predicted_labels = prediction.round()
        correct = (predicted_labels == labels_batch).sum().item()
        epoch_accuracy += correct

    # 1 Epoch 종료 후 평균 손실과 정확도 계산
    avg_loss = epoch_loss / len(data_loader)
    avg_accuracy = epoch_accuracy / len(dataset) # (총 맞힌 개수 / 총 샘플 개수)

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1:3d}/{num_epochs}] | Loss: {avg_loss:.4f} | Accuracy: {avg_accuracy * 100:.2f}%")

print("--- 학습 완료 ---")

# --- 5. (간단한 테스트) ---
# 학습된 모델로 아무 데이터나 예측해보기
# (주의: 실제로는 10일차에 배울 '데이터 전처리'를 똑같이 해줘야 함)
test_sample = torch.tensor([[0.9, 0.5, 10.0]], dtype=torch.float32).to(device) # [전세가율, 근저당, 10년차]
risk_prob = model(test_sample)
print(f"\n테스트 샘플 [0.9, 0.5, 10.0]의 위험 확률: {risk_prob.item() * 100:.2f}%")

Using device: cuda
--- 학습 시작 ---
Epoch [  5/50] | Loss: 0.6851 | Accuracy: 55.00%
Epoch [ 10/50] | Loss: 0.6813 | Accuracy: 56.00%
Epoch [ 15/50] | Loss: 0.6897 | Accuracy: 54.00%
Epoch [ 20/50] | Loss: 0.6778 | Accuracy: 50.00%
Epoch [ 25/50] | Loss: 0.6743 | Accuracy: 56.00%
Epoch [ 30/50] | Loss: 0.6685 | Accuracy: 56.00%
Epoch [ 35/50] | Loss: 0.6921 | Accuracy: 57.00%
Epoch [ 40/50] | Loss: 0.6768 | Accuracy: 56.00%
Epoch [ 45/50] | Loss: 0.6837 | Accuracy: 56.00%
Epoch [ 50/50] | Loss: 0.6848 | Accuracy: 52.00%
--- 학습 완료 ---

테스트 샘플 [0.9, 0.5, 10.0]의 위험 확률: 57.29%
